In [1]:
import sys
sys.path.append('../')
from model.VQAE import VQAE
from model.MARL import MARL
from utils import device, add_noise
from tqdm import tqdm
import torch
from data.FloorPlanLoader import *
import torch.nn.functional as F
import random
import json
import torchvision
import matplotlib.pyplot as plt
import geopandas as gpd

USE_MULTISCALE = True
USE_MULTITASK = True

#Reproducability Checks:
random.seed(0) #Python
torch.manual_seed(0) #Torch
np.random.seed(0) #NumPy

import shutil
import torchvision
import torchvision.transforms.functional as TF
import imageio
from torchvision.utils import save_image
# %load_ext autoreload
# %autoreload 2
import csv
import numpy
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd
import seaborn as sns
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [2]:
#Hyperparameter
device = torch.device('cpu')
batch_size = 128
n_hiddens = 32
n_residual_hiddens = 32
n_residual_layers = 1
embedding_dim = 64
n_embeddings = 218
beta = .25
lr = 3e-3
epochs = 100
noise=False
noise_weight=0.05
img_channel=3 if USE_MULTISCALE else 1

In [16]:
curr_data = 'data20_left'
n_clusters= 5

latent_file_path = f'../data/data_root/marl_latent_{curr_data}.pt'
latents = torch.load(latent_file_path)
floor = FloorPlanDataset(multi_scale=True, root=f'../data/data_root/{curr_data}/',\
                         data_config='../data/data_config/', preprocess=True)
data_loader = torch.utils.data.DataLoader(floor, batch_size=batch_size, shuffle=False)

In [3]:
import geopandas as gpd

In [21]:

# Replace 'path_to_json_file.json' with the actual path to your JSON file.
gdf = gpd.read_file('../data/data_root/geojson_40.json')


In [22]:
curr_data = 'data40'
gdf.head(2)

,OBJECTID,CODE,BLD_ID,YearBuilt1,HEIGHT,ELEV,footprint,UseType,UseDescription,Shape_Length_1,Shape_Area_1,geometry
0,1044491,Building,554863807836,1951,13.39,269.65,403.0,Residential,Single,331.111026,6822.16135,"MULTIPOLYGON (((-118.02249 33.96003, -118.0224..."
1,1044492,Building,526895829762,1962,27.00,294.06,2156.0,Residential,Single,364.889367,6623.85475,"MULTIPOLYGON (((-118.11468 34.02029, -118.1147..."


In [23]:
# Assuming the 'OBJECTID' column exists in the GeoDataFrame 'gdf'.
objectid_df = gdf[['OBJECTID']]
objectid_df.head(2)

,OBJECTID
0,1044491
1,1044492


In [17]:
df_center_index = pd.read_csv(f'../results/MARL_{curr_data}_{n_clusters}clusters_clusterindex.csv', index_col=None)
df_center_index

,label,index
0,3,523381
1,2,523382
2,2,523383
3,0,523384
4,0,523385
...,...,...
13326,3,549727
13327,0,549728
13328,1,549729
13329,2,549730


In [11]:
df_center_index_VQAE = pd.read_csv(f'../results/VQAE_{curr_data}_{n_clusters}clusters_clusterindex.csv', index_col=None)
df_center_index_VQAE

,label,index
0,0,1000
1,1,1001
2,4,1006
3,0,1007
4,4,1008
...,...,...
13972,0,993
13973,2,996
13974,1,997
13975,2,998


In [7]:
metadata_file_path = r"..\data\data_root\building_metadata_clean.csv"
metadata_df = pd.read_csv(metadata_file_path, index_col=False)
metadata_df.head(5)

,OBJECTID,vintage,idf.kw,FootprintArea.m2,building.area.m2,HEIGHT
0,19,1981-2008 (2004),SingleFamily-2004,302.896371,315.9529,22.87
1,20,1981-2008 (2004),SingleFamily-2004,91.815805,153.5637,25.52
2,21,before 1980,SingleFamily-pre-1980,176.129465,200.8498,22.21
3,22,before 1980,SingleFamily-pre-1980,113.052927,153.0992,28.80
4,23,1981-2008 (2004),SingleFamily-2004,108.517313,152.4489,22.32


In [24]:
merged_df = pd.merge(objectid_df, metadata_df, left_on='OBJECTID', right_on = 'OBJECTID', how='inner')
print(objectid_df.shape, merged_df.shape)
merged_df

(23265, 1) (11767, 6)


,OBJECTID,vintage,idf.kw,FootprintArea.m2,building.area.m2,HEIGHT
0,1044495,before 1980,SingleFamily-pre-1980,298.585281,180.8763,16.03
1,1044496,before 1980,SingleFamily-pre-1980,471.876042,261.1419,18.25
2,1044499,before 1980,SingleFamily-pre-1980,213.703678,146.5962,18.00
3,1044500,before 1980,SingleFamily-pre-1980,165.464074,83.8887,13.93
4,1044502,before 1980,SingleFamily-pre-1980,222.014143,130.6174,15.59
...,...,...,...,...,...,...
11762,1070334,1981-2008 (2004),SingleFamily-2004,19.673150,313.9091,4.82
11763,1070335,1981-2008 (2004),SingleFamily-2004,183.467832,190.5379,24.67
11764,1070336,before 1980,SingleFamily-pre-1980,101.208926,59.7347,14.13
11765,1070347,before 1980,SingleFamily-pre-1980,189.821967,106.2776,10.85


In [11]:
merged_df['idf.kw'].unique()

array(['SingleFamily-pre-1980', 'SingleFamily-2004',
       'MultiFamily-pre-1980', 'MultiFamily-2004', 'SingleFamily-2013',
       'MultiFamily-2013'], dtype=object)

In [25]:
# geojson 4
# Separate 'SingleFamily' values
single_family_df = merged_df[merged_df['idf.kw'].str.contains('SingleFamily')]
print(single_family_df.shape)
# Separate 'MultiFamily' values
multi_family_df = merged_df[merged_df['idf.kw'].str.contains('MultiFamily')]
print(multi_family_df.shape)

(11419, 6)
(348, 6)


In [9]:
# 8 
# Separate 'SingleFamily' values
single_family_df = merged_df[merged_df['idf.kw'].str.contains('SingleFamily')]
print(single_family_df.shape)
# Separate 'MultiFamily' values
multi_family_df = merged_df[merged_df['idf.kw'].str.contains('MultiFamily')]
print(multi_family_df.shape)

(7651, 6)
(2224, 6)


In [11]:
single_family_df

,OBJECTID,vintage,idf.kw,FootprintArea.m2,building.area.m2,HEIGHT
0,2099603,1981-2008 (2004),SingleFamily-2004,307.331319,217.5718,18.11
1,2099604,1981-2008 (2004),SingleFamily-2004,158.310718,199.3634,26.45
2,2099606,before 1980,SingleFamily-pre-1980,232.630210,150.3122,16.71
3,2099607,1981-2008 (2004),SingleFamily-2004,320.681808,317.5322,25.58
4,2099608,before 1980,SingleFamily-pre-1980,211.345775,137.9565,18.18
...,...,...,...,...,...,...
9863,2125005,before 1980,SingleFamily-pre-1980,118.685385,87.8834,NaN
9865,2125007,before 1980,SingleFamily-pre-1980,220.225660,71.3472,NaN
9868,2125014,before 1980,SingleFamily-pre-1980,202.558127,171.4005,NaN
9869,2125015,before 1980,SingleFamily-pre-1980,173.608631,135.1695,NaN


In [26]:
single_family_df[['OBJECTID']].to_csv(f'../notebooks/categorical_clustering/{curr_data}_singlefamily.csv')
multi_family_df[['OBJECTID']].to_csv(f'../notebooks/categorical_clustering/{curr_data}_multifamily.csv')

In [29]:
csv_file = f'../notebooks/categorical_clustering/{curr_data}_multifamily.csv'
# Read the CSV file into a dataframe
df = pd.read_csv(csv_file)
df

,Unnamed: 0,OBJECTID
0,8,1044510
1,17,1044534
2,22,1044550
3,57,1044609
4,62,1044619
...,...,...
343,11681,1070091
344,11698,1070151
345,11700,1070153
346,11702,1070166


In [31]:

# Assuming your CSV file has a column named 'OBJECTID' and the images are in the format 'f{objectid}.png'
curr_data_folder = f'../data/data_root/{curr_data}_SF/'
output_folder = f'../data/data_root/{curr_data}_MF/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
num_files_copied = 0

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    object_id = row['OBJECTID']
    image_file = f'{object_id}.png'
    pt_file = f'{object_id}.pt'
    img_source_path = os.path.join(curr_data_folder, image_file)
    img_destination_path = os.path.join(output_folder, image_file)
    
    pt_source_path = os.path.join(curr_data_folder, pt_file)
    pt_destination_path = os.path.join(output_folder, pt_file)
    
    if os.path.exists(img_source_path):
        shutil.move(img_source_path, img_destination_path)
        shutil.move(pt_source_path, pt_destination_path)
        num_files_copied += 1

# Display the total number of files copied
print(f"Total number of files copied: {num_files_copied}")

Total number of files copied: 348
